<table style="width:100%">
<tr>
<td style="vertical-align:middle; text-align:left;">
<font size="2">
Supplementary code for the <a href="http://mng.bz/orYv">Build a Large Language Model From Scratch</a> book by <a href="https://sebastianraschka.com">Sebastian Raschka</a><br>
<br>Code repository: <a href="https://github.com/rasbt/LLMs-from-scratch">https://github.com/rasbt/LLMs-from-scratch</a>
</font>
</td>
<td style="vertical-align:middle; text-align:left;">
<a href="http://mng.bz/orYv"><img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/cover-small.webp" width="100px"></a>
</td>
</tr>
</table>


# Capítulo 3: Codificando Attention Mechanisms

Packages that are being used in this notebook:

In [ ]:
from importlib.metadata import version

print("torch version:", version("torch"))

torch version: 2.4.0


- Este capítulo aborda os mecanismos de atenção, o motor das LLMs.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/01.webp?123" width="500px">

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/02.webp" width="600px">

## 3.1 O problema ao modelar sequências longas

- Nenhum código nesta seção.  
- Traduzir um texto palavra por palavra não é viável devido às diferenças nas estruturas gramaticais entre a língua de origem e a língua de destino.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/03.webp" width="400px">

- Antes da introdução dos modelos Transformer, RNNs encoder-decoder eram comumente usadas para tarefas de tradução automática.  
- Nesse modelo, o encoder processa uma sequência de tokens do idioma de origem, utilizando um estado oculto — um tipo de camada intermediária dentro da rede neural — para gerar uma representação condensada de toda a sequência de entrada.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/04.webp" width="500px">

## 3.2 Capturando dependências nos dados com mecanismos de atenção

- Nenhum código nesta seção.  
- Por meio de um mecanismo de atenção, a parte decodificadora da rede, responsável pela geração de texto, é capaz de acessar seletivamente todos os tokens de entrada, o que implica que certos tokens de entrada possuem mais importância do que outros na geração de um token de saída específico.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/05.webp" width="500px">

- A self-attention em transformers é uma técnica projetada para aprimorar as representações de entrada, permitindo que cada posição em uma sequência interaja com e determine a relevância de todas as outras posições dentro da mesma sequência.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/06.webp" width="300px">

## 3.3 Atenção a diferentes partes da entrada com autoatenção

### 3.3.1 Um mecanismo simples de autoatenção sem pesos treináveis

- Esta seção explica uma variante muito simplificada de autoatenção, que não contém pesos treináveis.  
- Isso é apenas para fins ilustrativos e NÃO é o mecanismo de atenção usado nos transformers.  
- A próxima seção, seção 3.3.2, ampliará esse mecanismo simples para implementar o mecanismo real de autoatenção.  

- Suponha que tenhamos uma sequência de entrada \( x^{(1)} \) a \( x^{(T)} \):  
  - A entrada é um texto (por exemplo, uma frase como "Sua jornada começa com um passo") que já foi convertida em embeddings de tokens, conforme descrito no capítulo 2.  
  - Por exemplo, \( x^{(1)} \) é um vetor de dimensão \( d \) representando a palavra "Sua", e assim por diante.  

- **Objetivo:** calcular os vetores de contexto \( z^{(i)} \) para cada elemento da sequência de entrada \( x^{(i)} \) em \( x^{(1)} \) a \( x^{(T)} \) (onde \( z \) e \( x \) têm a mesma dimensão).  
  - Um vetor de contexto \( z^{(i)} \) é uma soma ponderada das entradas \( x^{(1)} \) a \( x^{(T)} \).  
  - O vetor de contexto é específico para cada entrada.  
    - Em vez de considerar \( x^{(i)} \) como um marcador genérico para qualquer token de entrada, vamos analisar o segundo token de entrada, \( x^{(2)} \).  
    - E, para um exemplo mais concreto, em vez de \( z^{(i)} \), consideramos o segundo vetor de contexto de saída, \( z^{(2)} \).  
    - O vetor de contexto \( z^{(2)} \) é uma soma ponderada de todas as entradas \( x^{(1)} \) a \( x^{(T)} \), ponderadas em relação ao segundo elemento de entrada, \( x^{(2)} \).  
    - Os pesos de atenção determinam quanto cada um dos elementos de entrada contribui para a soma ponderada ao calcular \( z^{(2)} \).  
    - Resumindo, podemos pensar em \( z^{(2)} \) como uma versão modificada de \( x^{(2)} \) que também incorpora informações sobre outros elementos da entrada que são relevantes para a tarefa em questão.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/07.webp" width="400px">

- (Observe que os números nesta figura foram truncados para uma casa decimal após o ponto para reduzir a poluição visual; da mesma forma, outras figuras também podem conter valores truncados)

- Por convenção, os pesos de atenção não normalizados são chamados de **"attention scores"**, enquanto os scores de atenção normalizados, que somam 1, são chamados de **"attention weights"**.

- O código abaixo percorre a figura acima passo a passo

<br>

- Passo 1: calcular os scores de atenção não normalizados 𝜔

- Suponha que usamos o segundo token de entrada como a consulta (query), ou seja, $q^{(2)} = x^{(2)}$. Calculamos os scores de atenção não normalizados usando produtos escalares:
    - $\omega_{21} = x^{(1)} q^{(2)\top}$
    - $\omega_{22} = x^{(2)} q^{(2)\top}$
    - $\omega_{23} = x^{(3)} q^{(2)\top}$
    - ...
    - $\omega_{2T} = x^{(T)} q^{(2)\top}$
- Acima, $\omega$ é a letra grega "ômega", usada para simbolizar os scores de atenção não normalizados.
    - O subíndice "21" em $\omega_{21}$ significa que o segundo elemento da sequência de entrada foi usado como consulta (query) contra o primeiro elemento da sequência de entrada.

- Suponha que temos a seguinte sentença de entrada, que já foi incorporada em vetores tridimensionais, conforme descrito no capítulo 3 (aqui usamos uma dimensão de embedding muito pequena apenas para fins ilustrativos, para que caiba na página sem quebras de linha).

In [ ]:
import torch

inputs = torch.tensor(
  [[0.43, 0.15, 0.89], # Your     (x^1)
   [0.55, 0.87, 0.66], # journey  (x^2)
   [0.57, 0.85, 0.64], # starts   (x^3)
   [0.22, 0.58, 0.33], # with     (x^4)
   [0.77, 0.25, 0.10], # one      (x^5)
   [0.05, 0.80, 0.55]] # step     (x^6)
)

- (Neste livro, seguimos a convenção comum de aprendizado de máquina e aprendizado profundo, onde os exemplos de treinamento são representados como linhas e os valores das características como colunas; no caso do tensor mostrado acima, cada linha representa uma palavra e cada coluna representa uma dimensão do embedding).  

- O principal objetivo desta seção é demonstrar como o vetor de contexto $z^{(2)}$  é calculado usando a segunda sequência de entrada, $x^{(2)}$, como consulta (*query*).  

- A figura ilustra a etapa inicial desse processo, que envolve o cálculo dos scores de atenção ω entre $x^{(2)}$ e todos os outros elementos de entrada por meio de uma operação de produto escalar.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/08.webp" width="400px">

- Usamos o elemento 2 da sequência de entrada, $x^{(2)}$, como exemplo para calcular o vetor de contexto $z^{(2)}$; mais adiante nesta seção, iremos generalizar esse processo para calcular todos os vetores de contexto.  
- O primeiro passo é calcular os scores de atenção não normalizados realizando o produto escalar entre a consulta $x^{(2)}$ e todos os outros tokens de entrada.

In [ ]:
query = inputs[1]  # O segundo token de entrada é a consulta

attn_scores_2 = torch.empty(inputs.shape[0])
for i, x_i in enumerate(inputs):
    attn_scores_2[i] = torch.dot(x_i, query) # produto escalar (transposição não é necessária aqui, pois são vetores unidimensionais)

print(attn_scores_2)

tensor([0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865])


- Observação: um produto escalar é essencialmente uma forma simplificada de multiplicar dois vetores elemento por elemento e somar os produtos resultantes.

In [ ]:
res = 0.

for idx, element in enumerate(inputs[0]):
    res += inputs[0][idx] * query[idx]

print(res)
print(torch.dot(inputs[0], query))

tensor(0.9544)
tensor(0.9544)


- Passo 2: normalizar os scores de atenção não normalizados ("ômega", $\omega$) para que sua soma seja igual a 1.  
- Aqui está uma maneira simples de normalizar os scores de atenção não normalizados para que somem 1 (uma convenção útil para interpretação e importante para a estabilidade do treinamento).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/09.webp" width="500px">

In [ ]:
attn_weights_2_tmp = attn_scores_2 / attn_scores_2.sum()

print("Attention weights:", attn_weights_2_tmp)
print("Sum:", attn_weights_2_tmp.sum())

Attention weights: tensor([0.1455, 0.2278, 0.2249, 0.1285, 0.1077, 0.1656])
Sum: tensor(1.0000)


- No entanto, na prática, é comum e recomendado usar a função softmax para normalização, pois ela lida melhor com valores extremos e possui propriedades de gradiente mais desejáveis durante o treinamento.  
- Aqui está uma implementação simples da função softmax para escalonamento, que também normaliza os elementos do vetor para que sua soma seja igual a 1:

In [ ]:
def softmax_naive(x):
    return torch.exp(x) / torch.exp(x).sum(dim=0)

attn_weights_2_naive = softmax_naive(attn_scores_2)

print("Attention weights:", attn_weights_2_naive)
print("Sum:", attn_weights_2_naive.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- A implementação ingênua acima pode sofrer com problemas de instabilidade numérica para valores de entrada muito grandes ou muito pequenos, devido a problemas de overflow e underflow.  
- Portanto, na prática, é recomendado usar a implementação do softmax do PyTorch, que foi altamente otimizada para desempenho.

In [ ]:
attn_weights_2 = torch.softmax(attn_scores_2, dim=0)

print("Attention weights:", attn_weights_2)
print("Sum:", attn_weights_2.sum())

Attention weights: tensor([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
Sum: tensor(1.)


- Passo 3: calcular o vetor de contexto $z^{(2)}$ multiplicando os tokens de entrada incorporados, $x^{(i)}$, pelos pesos de atenção e somando os vetores resultantes.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/10.webp" width="500px">

In [ ]:
query = inputs[1] # 2nd input token is the query

context_vec_2 = torch.zeros(query.shape)
for i,x_i in enumerate(inputs):
    context_vec_2 += attn_weights_2[i]*x_i

print(context_vec_2)

tensor([0.4419, 0.6515, 0.5683])


### 3.3.2 Calculando os pesos de atenção para todos os tokens de entrada

#### Generalizar para todos os tokens da sequência de entrada:  

- Acima, calculamos os pesos de atenção e o vetor de contexto para o segundo token de entrada (como ilustrado na linha destacada na figura abaixo).  
- Agora, estamos generalizando esse cálculo para computar todos os pesos de atenção e vetores de contexto.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/11.webp" width="400px">

- (Please note that the numbers in this figure are truncated to two
digits after the decimal point to reduce visual clutter; the values in each row should add up to 1.0 or 100%; similarly, digits in other figures are truncated)

- Na autoatenção, o processo começa com o cálculo dos scores de atenção, que são posteriormente normalizados para obter os pesos de atenção, cuja soma totaliza 1.  
- Esses pesos de atenção são então utilizados para gerar os vetores de contexto por meio de uma soma ponderada das entradas.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/12.webp" width="400px">

- Aplicar o **passo 1** anterior a todos os pares de elementos para calcular a matriz de scores de atenção não normalizados.

In [ ]:
attn_scores = torch.empty(6, 6)

for i, x_i in enumerate(inputs):
    for j, x_j in enumerate(inputs):
        attn_scores[i, j] = torch.dot(x_i, x_j)

print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- Podemos obter o mesmo resultado de forma mais eficiente por meio da multiplicação de matrizes.

In [ ]:
attn_scores = inputs @ inputs.T
print(attn_scores)

tensor([[0.9995, 0.9544, 0.9422, 0.4753, 0.4576, 0.6310],
        [0.9544, 1.4950, 1.4754, 0.8434, 0.7070, 1.0865],
        [0.9422, 1.4754, 1.4570, 0.8296, 0.7154, 1.0605],
        [0.4753, 0.8434, 0.8296, 0.4937, 0.3474, 0.6565],
        [0.4576, 0.7070, 0.7154, 0.3474, 0.6654, 0.2935],
        [0.6310, 1.0865, 1.0605, 0.6565, 0.2935, 0.9450]])


- Similar ao **passo 2** anterior, normalizamos cada linha para que os valores em cada linha somem 1.

In [ ]:
attn_weights = torch.softmax(attn_scores, dim=-1)
print(attn_weights)

tensor([[0.2098, 0.2006, 0.1981, 0.1242, 0.1220, 0.1452],
        [0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581],
        [0.1390, 0.2369, 0.2326, 0.1242, 0.1108, 0.1565],
        [0.1435, 0.2074, 0.2046, 0.1462, 0.1263, 0.1720],
        [0.1526, 0.1958, 0.1975, 0.1367, 0.1879, 0.1295],
        [0.1385, 0.2184, 0.2128, 0.1420, 0.0988, 0.1896]])


- Verificação rápida para confirmar que os valores em cada linha realmente somam 1.

In [ ]:
row_2_sum = sum([0.1385, 0.2379, 0.2333, 0.1240, 0.1082, 0.1581])
print("Row 2 sum:", row_2_sum)

print("All row sums:", attn_weights.sum(dim=-1))

Row 2 sum: 1.0
All row sums: tensor([1.0000, 1.0000, 1.0000, 1.0000, 1.0000, 1.0000])


- Aplicar o **passo 3** anterior para calcular todos os vetores de contexto.

In [ ]:
all_context_vecs = attn_weights @ inputs
print(all_context_vecs)

tensor([[0.4421, 0.5931, 0.5790],
        [0.4419, 0.6515, 0.5683],
        [0.4431, 0.6496, 0.5671],
        [0.4304, 0.6298, 0.5510],
        [0.4671, 0.5910, 0.5266],
        [0.4177, 0.6503, 0.5645]])


- Como uma verificação de consistência, o vetor de contexto previamente calculado $z^{(2)} = [0.4419, 0.6515, 0.5683]$ pode ser encontrado na segunda linha acima.

In [ ]:
print("Previous 2nd context vector:", context_vec_2)

Previous 2nd context vector: tensor([0.4419, 0.6515, 0.5683])


## 3.4 Implementando autoatenção com pesos treináveis

- Um modelo conceitual ilustrando como o mecanismo de autoatenção desenvolvido nesta seção se integra à narrativa e estrutura geral deste livro e capítulo.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/13.webp" width="400px">

### 3.4.1 Calculando os pesos de atenção passo a passo

Nesta seção, estamos implementando o mecanismo de autoatenção utilizado na arquitetura original dos transformers, nos modelos GPT e na maioria das LLMs populares.  
Esse mecanismo de autoatenção também é chamado de "atenção por produto escalar escalado" (*scaled dot-product attention*).  

A ideia geral é semelhante à apresentada anteriormente:  
- Queremos calcular vetores de contexto como somas ponderadas dos vetores de entrada, específicos para um determinado elemento da sequência.  
- Para isso, precisamos de pesos de atenção.  

Como veremos, há apenas pequenas diferenças em relação ao mecanismo de atenção básico introduzido anteriormente:  
- A diferença mais notável é a introdução de **matrizes de pesos** que são atualizadas durante o treinamento do modelo.  
- Essas **matrizes de pesos treináveis** são fundamentais para que o modelo (especificamente o módulo de atenção dentro do modelo) possa aprender a produzir **vetores de contexto "bons"**.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/14.webp" width="600px">

- Implementando o mecanismo de autoatenção passo a passo, começaremos introduzindo as três matrizes de pesos treináveis. $W_q$, $W_k$, e $W_v$
- Implementando o mecanismo de autoatenção passo a passo, começaremos introduzindo as três matrizes de pesos treináveis. $x^{(i)}$, em query, key, and value vectors via matrix multiplication:

  - Query vector: $q^{(i)} = W_q \,x^{(i)}$
  - Key vector: $k^{(i)} = W_k \,x^{(i)}$
  - Value vector: $v^{(i)} = W_v \,x^{(i)}$


- As dimensões de embedding da entrada $x$ e do vetor de consulta $q$ podem ser iguais ou diferentes, dependendo do design e da implementação específica do modelo.  
- Nos modelos GPT, as dimensões de entrada e saída geralmente são as mesmas, mas, para fins ilustrativos e para melhor acompanhar os cálculos, escolhemos dimensões de entrada e saída diferentes aqui.

In [ ]:
x_2 = inputs[1] # second input element
d_in = inputs.shape[1] # the input embedding size, d=3
d_out = 2 # the output embedding size, d=2

- Abaixo, inicializamos as três matrizes de pesos; observe que estamos definindo `requires_grad=False` para reduzir a complexidade das saídas para fins ilustrativos, mas, se fôssemos usar essas matrizes de pesos para o treinamento do modelo, definiríamos `requires_grad=True` para que elas fossem atualizadas durante o treinamento.

In [ ]:
torch.manual_seed(123)

W_query = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_key   = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)
W_value = torch.nn.Parameter(torch.rand(d_in, d_out), requires_grad=False)

- Em seguida, calculamos os vetores de consulta (*query*), chave (*key*) e valor (*value*).

In [ ]:
query_2 = x_2 @ W_query # _2 because it's with respect to the 2nd input element
key_2 = x_2 @ W_key
value_2 = x_2 @ W_value

print(query_2)

tensor([0.4306, 1.4551])


- Como podemos ver abaixo, projetamos com sucesso os 6 tokens de entrada de um espaço de embedding 3D para um espaço de embedding 2D.

In [ ]:
keys = inputs @ W_key
values = inputs @ W_value

print("keys.shape:", keys.shape)
print("values.shape:", values.shape)

keys.shape: torch.Size([6, 2])
values.shape: torch.Size([6, 2])


- No próximo passo, **passo 2**, calculamos os scores de atenção não normalizados realizando o produto escalar entre a consulta (*query*) e cada vetor de chave (*key*).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/15.webp" width="600px">

In [ ]:
keys_2 = keys[1] # Python starts index at 0
attn_score_22 = query_2.dot(keys_2)
print(attn_score_22)

tensor(1.8524)


- Como temos 6 entradas, obtemos 6 scores de atenção para o vetor de consulta fornecido.

In [ ]:
attn_scores_2 = query_2 @ keys.T # All attention scores for given query
print(attn_scores_2)

tensor([1.2705, 1.8524, 1.8111, 1.0795, 0.5577, 1.5440])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/16.webp" width="600px">

- Em seguida, no **passo 3**, calculamos os pesos de atenção (scores de atenção normalizados que somam 1) usando a função softmax que utilizamos anteriormente.  
- A diferença em relação ao que fizemos antes é que agora escalamos os scores de atenção dividindo-os pela raiz quadrada da dimensão do embedding, $\sqrt{d_k}$  (ou seja, `d_k**0.5`).

In [ ]:
d_k = keys.shape[1]
attn_weights_2 = torch.softmax(attn_scores_2 / d_k**0.5, dim=-1)
print(attn_weights_2)

tensor([0.1500, 0.2264, 0.2199, 0.1311, 0.0906, 0.1820])


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/17.webp" width="600px">

- No **passo 4**, agora calculamos o vetor de contexto para o vetor de consulta da entrada 2.

In [ ]:
context_vec_2 = attn_weights_2 @ values
print(context_vec_2)

tensor([0.3061, 0.8210])


### 3.4.2 Implementando uma classe compacta de SelfAttention

- Reunindo tudo, podemos implementar o mecanismo de autoatenção da seguinte forma:

In [ ]:
import torch.nn as nn

class SelfAttention_v1(nn.Module):

    def __init__(self, d_in, d_out):
        super().__init__()
        self.W_query = nn.Parameter(torch.rand(d_in, d_out))
        self.W_key   = nn.Parameter(torch.rand(d_in, d_out))
        self.W_value = nn.Parameter(torch.rand(d_in, d_out))

    def forward(self, x):
        keys = x @ self.W_key
        queries = x @ self.W_query
        values = x @ self.W_value

        attn_scores = queries @ keys.T # omega
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)
sa_v1 = SelfAttention_v1(d_in, d_out)
print(sa_v1(inputs))

tensor([[0.2996, 0.8053],
        [0.3061, 0.8210],
        [0.3058, 0.8203],
        [0.2948, 0.7939],
        [0.2927, 0.7891],
        [0.2990, 0.8040]], grad_fn=<MmBackward0>)


<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/18.webp" width="400px">

- Podemos simplificar a implementação acima utilizando as camadas **Linear** do PyTorch, que são equivalentes a uma multiplicação de matrizes quando desativamos as unidades de viés.  
- Outra grande vantagem de usar `nn.Linear` em vez da abordagem manual com `nn.Parameter(torch.rand(...))` é que `nn.Linear` possui um esquema de inicialização de pesos preferencial, o que resulta em um treinamento do modelo mais estável.

In [ ]:
class SelfAttention_v2(nn.Module):

    def __init__(self, d_in, d_out, qkv_bias=False):
        super().__init__()
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)

    def forward(self, x):
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.T
        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(789)
sa_v2 = SelfAttention_v2(d_in, d_out)
print(sa_v2(inputs))

tensor([[-0.0739,  0.0713],
        [-0.0748,  0.0703],
        [-0.0749,  0.0702],
        [-0.0760,  0.0685],
        [-0.0763,  0.0679],
        [-0.0754,  0.0693]], grad_fn=<MmBackward0>)


- Observe que `SelfAttention_v1` e `SelfAttention_v2` produzem saídas diferentes porque utilizam pesos iniciais distintos para as matrizes de pesos.

## 3.5 Ocultando palavras futuras com atenção causal

- Na atenção causal, os pesos de atenção acima da diagonal são mascarados, garantindo que, para qualquer entrada, a LLM não possa utilizar tokens futuros ao calcular os vetores de contexto com os pesos de atenção.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/19.webp" width="400px">

### 3.5.1 Aplicando uma máscara de atenção causal

- Nesta seção, estamos convertendo o mecanismo de autoatenção anterior em um mecanismo de autoatenção causal.  
- A autoatenção causal garante que a previsão do modelo para uma determinada posição em uma sequência dependa apenas das saídas conhecidas nas posições anteriores, e não em posições futuras.  
- Em outras palavras, isso assegura que cada previsão da próxima palavra dependa apenas das palavras anteriores.  
- Para alcançar esse objetivo, mascaramos os tokens futuros para cada token dado (ou seja, aqueles que vêm depois do token atual no texto de entrada).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/20.webp" width="600px">

- Para ilustrar e implementar a autoatenção causal, vamos trabalhar com os scores de atenção e pesos da seção anterior:

In [ ]:
# Reutilize as matrizes de pesos de consulta e chave do
# objeto SelfAttention_v2 da seção anterior para conveniência
queries = sa_v2.W_query(inputs)
keys = sa_v2.W_key(inputs)
attn_scores = queries @ keys.T

attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[0.1921, 0.1646, 0.1652, 0.1550, 0.1721, 0.1510],
        [0.2041, 0.1659, 0.1662, 0.1496, 0.1665, 0.1477],
        [0.2036, 0.1659, 0.1662, 0.1498, 0.1664, 0.1480],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.1661, 0.1564],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.1585],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


- A maneira mais simples de mascarar os pesos de atenção futuros é criando uma máscara usando a função `tril` do PyTorch, com os elementos abaixo da diagonal principal (incluindo a diagonal) definidos como 1 e acima da diagonal principal definidos como 0:

In [ ]:
context_length = attn_scores.shape[0]
mask_simple = torch.tril(torch.ones(context_length, context_length))
print(mask_simple)

tensor([[1., 0., 0., 0., 0., 0.],
        [1., 1., 0., 0., 0., 0.],
        [1., 1., 1., 0., 0., 0.],
        [1., 1., 1., 1., 0., 0.],
        [1., 1., 1., 1., 1., 0.],
        [1., 1., 1., 1., 1., 1.]])


- Em seguida, podemos multiplicar os pesos de atenção por essa máscara para zerar os scores de atenção acima da diagonal.

In [ ]:
masked_simple = attn_weights*mask_simple
print(masked_simple)

tensor([[0.1921, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2041, 0.1659, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.2036, 0.1659, 0.1662, 0.0000, 0.0000, 0.0000],
        [0.1869, 0.1667, 0.1668, 0.1571, 0.0000, 0.0000],
        [0.1830, 0.1669, 0.1670, 0.1588, 0.1658, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<MulBackward0>)


- No entanto, se a máscara fosse aplicada após o softmax, como acima, isso perturbaria a distribuição de probabilidades criada pelo softmax.  
- O softmax garante que todos os valores de saída somem 1.  
- Aplicar a máscara após o softmax exigiria re-normalizar as saídas para que somassem 1 novamente, o que complicaria o processo e poderia levar a efeitos indesejados.

- Para garantir que as linhas somem 1, podemos normalizar os pesos de atenção da seguinte forma:

In [ ]:
row_sums = masked_simple.sum(dim=-1, keepdim=True)
masked_simple_norm = masked_simple / row_sums
print(masked_simple_norm)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<DivBackward0>)


- Embora tecnicamente já tenhamos finalizado o código do mecanismo de atenção causal, vamos dar uma olhada brevemente em uma abordagem mais eficiente para alcançar o mesmo resultado que o acima.  
- Assim, em vez de zerar os pesos de atenção acima da diagonal e renormalizar os resultados, podemos mascarar os scores de atenção não normalizados acima da diagonal com **menos infinito** antes de entrarem na função softmax.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/21.webp" width="450px">

In [ ]:
mask = torch.triu(torch.ones(context_length, context_length), diagonal=1)
masked = attn_scores.masked_fill(mask.bool(), -torch.inf)
print(masked)

tensor([[0.2899,   -inf,   -inf,   -inf,   -inf,   -inf],
        [0.4656, 0.1723,   -inf,   -inf,   -inf,   -inf],
        [0.4594, 0.1703, 0.1731,   -inf,   -inf,   -inf],
        [0.2642, 0.1024, 0.1036, 0.0186,   -inf,   -inf],
        [0.2183, 0.0874, 0.0882, 0.0177, 0.0786,   -inf],
        [0.3408, 0.1270, 0.1290, 0.0198, 0.1290, 0.0078]],
       grad_fn=<MaskedFillBackward0>)


- Como podemos ver abaixo, agora os pesos de atenção em cada linha somam corretamente 1 novamente.

In [ ]:
attn_weights = torch.softmax(masked / keys.shape[-1]**0.5, dim=-1)
print(attn_weights)

tensor([[1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.5517, 0.4483, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.3800, 0.3097, 0.3103, 0.0000, 0.0000, 0.0000],
        [0.2758, 0.2460, 0.2462, 0.2319, 0.0000, 0.0000],
        [0.2175, 0.1983, 0.1984, 0.1888, 0.1971, 0.0000],
        [0.1935, 0.1663, 0.1666, 0.1542, 0.1666, 0.1529]],
       grad_fn=<SoftmaxBackward0>)


### 3.5.2 Mascarando pesos de atenção adicionais com dropout

- Além disso, também aplicamos dropout para reduzir o overfitting durante o treinamento.  
- O dropout pode ser aplicado em vários lugares:  
  - por exemplo, após calcular os pesos de atenção;  
  - ou após multiplicar os pesos de atenção pelos vetores de valor.  
- Aqui, aplicaremos a máscara de dropout após calcular os pesos de atenção, pois isso é mais comum.  

- Além disso, neste exemplo específico, usamos uma taxa de dropout de 50%, o que significa mascarar aleatoriamente metade dos pesos de atenção. (Quando treinarmos o modelo GPT mais tarde, usaremos uma taxa de dropout mais baixa, como 0,1 ou 0,2).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/22.webp" width="400px">

- Se aplicarmos uma taxa de dropout de 0,5 (50%), os valores não descartados serão escalados proporcionalmente por um fator de 1/0,5 = 2.  
- O escalonamento é calculado pela fórmula 1 / (1 - `dropout_rate`).

In [ ]:
torch.manual_seed(123)
dropout = torch.nn.Dropout(0.5) # dropout rate of 50%
example = torch.ones(6, 6) # create a matrix of ones

print(dropout(example))

tensor([[2., 2., 0., 2., 2., 0.],
        [0., 0., 0., 2., 0., 2.],
        [2., 2., 2., 2., 0., 2.],
        [0., 2., 2., 0., 0., 2.],
        [0., 2., 0., 2., 0., 2.],
        [0., 2., 2., 2., 2., 0.]])


In [ ]:
torch.manual_seed(123)
print(dropout(attn_weights))

tensor([[2.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000],
        [0.7599, 0.6194, 0.6206, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.4921, 0.4925, 0.0000, 0.0000, 0.0000],
        [0.0000, 0.3966, 0.0000, 0.3775, 0.0000, 0.0000],
        [0.0000, 0.3327, 0.3331, 0.3084, 0.3331, 0.0000]],
       grad_fn=<MulBackward0>)


- Observe que as saídas resultantes do dropout podem parecer diferentes dependendo do seu sistema operacional; você pode ler mais sobre essa inconsistência [aqui no rastreador de problemas do PyTorch](https://github.com/pytorch/pytorch/issues/121595).

### 3.5.3 Implementando uma classe compacta de autoatenção causal

- Agora, estamos prontos para implementar uma versão funcional de autoatenção, incluindo as máscaras de causalidade e dropout.  
- Uma coisa a mais é implementar o código para lidar com lotes (batches) consistindo em mais de uma entrada, de modo que nossa classe `CausalAttention` suporte as saídas em lote produzidas pelo data loader que implementamos no capítulo 2.  
- Para simplificar, para simular tal entrada em lote, duplicamos o exemplo de texto de entrada.

In [ ]:
batch = torch.stack((inputs, inputs), dim=0)
print(batch.shape) # 2 inputs with 6 tokens each, and each token has embedding dimension 3

torch.Size([2, 6, 3])


In [ ]:
class CausalAttention(nn.Module):

    def __init__(self, d_in, d_out, context_length,
                 dropout, qkv_bias=False):
        super().__init__()
        self.d_out = d_out
        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key   = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.dropout = nn.Dropout(dropout) # New
        self.register_buffer('mask', torch.triu(torch.ones(context_length, context_length), diagonal=1)) # New

    def forward(self, x):
        b, num_tokens, d_in = x.shape # Nova dimensão de lote b
        # Para entradas onde `num_tokens` excede `context_length`, isso resultará em erros
        # na criação da máscara abaixo.
        # Na prática, isso não é um problema, pois o LLM (capítulos 4-7) garante que as entradas
        # não excedam `context_length` antes de chegar a este método forward.
        keys = self.W_key(x)
        queries = self.W_query(x)
        values = self.W_value(x)

        attn_scores = queries @ keys.transpose(1, 2) # Changed transpose
        attn_scores.masked_fill_(  # New, _ ops are in-place
            self.mask.bool()[:num_tokens, :num_tokens], -torch.inf)  # `:num_tokens` para levar em conta os casos em que o número de tokens no lote é menor do que o tamanho de contexto suportado
        attn_weights = torch.softmax(
            attn_scores / keys.shape[-1]**0.5, dim=-1
        )
        attn_weights = self.dropout(attn_weights) # New

        context_vec = attn_weights @ values
        return context_vec

torch.manual_seed(123)

context_length = batch.shape[1]
ca = CausalAttention(d_in, d_out, context_length, 0.0)

context_vecs = ca(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]],

        [[-0.4519,  0.2216],
         [-0.5874,  0.0058],
         [-0.6300, -0.0632],
         [-0.5675, -0.0843],
         [-0.5526, -0.0981],
         [-0.5299, -0.1081]]], grad_fn=<UnsafeViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Observe que o dropout é aplicado apenas durante o treinamento, não durante a inferência.

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/23.webp" width="500px">

## 3.6 Estendendo a atenção de cabeça única para atenção multi-cabeça

### 3.6.1 Empilhando várias camadas de atenção de cabeça única

- Abaixo está um resumo da autoatenção implementada anteriormente (máscaras de causalidade e dropout não mostradas por simplicidade).  

- Isso também é chamado de atenção de cabeça única:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/24.webp" width="400px">

- Simplesmente empilhamos múltiplos módulos de atenção de cabeça única para obter um módulo de atenção multi-cabeça:

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/25.webp" width="400px">

- A ideia principal por trás da atenção multi-cabeça é executar o mecanismo de atenção várias vezes (em paralelo) com diferentes projeções lineares aprendidas. Isso permite que o modelo preste atenção conjunta em informações de diferentes subespaços de representação em diferentes posições.

In [ ]:
class MultiHeadAttentionWrapper(nn.Module):

    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        self.heads = nn.ModuleList(
            [CausalAttention(d_in, d_out, context_length, dropout, qkv_bias)
             for _ in range(num_heads)]
        )

    def forward(self, x):
        return torch.cat([head(x) for head in self.heads], dim=-1)


torch.manual_seed(123)

context_length = batch.shape[1] # This is the number of tokens
d_in, d_out = 3, 2
mha = MultiHeadAttentionWrapper(
    d_in, d_out, context_length, 0.0, num_heads=2
)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]],

        [[-0.4519,  0.2216,  0.4772,  0.1063],
         [-0.5874,  0.0058,  0.5891,  0.3257],
         [-0.6300, -0.0632,  0.6202,  0.3860],
         [-0.5675, -0.0843,  0.5478,  0.3589],
         [-0.5526, -0.0981,  0.5321,  0.3428],
         [-0.5299, -0.1081,  0.5077,  0.3493]]], grad_fn=<CatBackward0>)
context_vecs.shape: torch.Size([2, 6, 4])


- Na implementação acima, a dimensão do embedding é 4, porque definimos `d_out=2` como a dimensão do embedding para os vetores de chave, consulta e valor, assim como para o vetor de contexto. E como temos 2 cabeças de atenção, a dimensão do embedding de saída é $2*2=4$

### 3.6.2 Implementando atenção multi-cabeça com divisões de peso

- Embora a implementação acima seja uma implementação intuitiva e totalmente funcional da atenção multi-cabeça (envolvendo a implementação de atenção de cabeça única `CausalAttention` mencionada anteriormente), podemos escrever uma classe independente chamada `MultiHeadAttention` para alcançar o mesmo resultado.

- Não concatenamos as cabeças de atenção únicas para esta classe independente `MultiHeadAttention`.  
- Em vez disso, criamos matrizes de pesos W_query, W_key e W_value únicas e, em seguida, dividimos essas matrizes em matrizes individuais para cada cabeça de atenção.

In [ ]:
class MultiHeadAttention(nn.Module):
    def __init__(self, d_in, d_out, context_length, dropout, num_heads, qkv_bias=False):
        super().__init__()
        assert (d_out % num_heads == 0), \
            "d_out must be divisible by num_heads"

        self.d_out = d_out
        self.num_heads = num_heads
        self.head_dim = d_out // num_heads # Reduzir a dimensão da projeção para coincidir com a dimensão de saída desejada

        self.W_query = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_key = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.W_value = nn.Linear(d_in, d_out, bias=qkv_bias)
        self.out_proj = nn.Linear(d_out, d_out)  # Linear layer to combine head outputs
        self.dropout = nn.Dropout(dropout)
        self.register_buffer(
            "mask",
            torch.triu(torch.ones(context_length, context_length),
                       diagonal=1)
        )

    def forward(self, x):
        b, num_tokens, d_in = x.shape
        # Como em CausalAttention, para entradas onde num_tokens excede context_length,
        # isso resultará em erros na criação da máscara mais abaixo.
        # Na prática, isso não é um problema, pois o LLM (capítulos 4-7) garante que as entradas
        # não excedam `context_length` antes de chegar a este método forward.


        keys = self.W_key(x) # Shape: (b, num_tokens, d_out)
        queries = self.W_query(x)
        values = self.W_value(x)

        # Dividimos implicitamente a matriz adicionando uma dimensão num_heads
        # Desenrolar a última dimensão: (b, num_tokens, d_out) -> (b, num_tokens, num_heads, head_dim)
        keys = keys.view(b, num_tokens, self.num_heads, self.head_dim)
        values = values.view(b, num_tokens, self.num_heads, self.head_dim)
        queries = queries.view(b, num_tokens, self.num_heads, self.head_dim)

        # Transpose: (b, num_tokens, num_heads, head_dim) -> (b, num_heads, num_tokens, head_dim)
        keys = keys.transpose(1, 2)
        queries = queries.transpose(1, 2)
        values = values.transpose(1, 2)

        # Calcular a atenção por produto escalar escalado (também conhecida como autoatenção) com uma máscara causal
        attn_scores = queries @ keys.transpose(2, 3)  # Dot product for each head

        # Máscara original truncada para o número de tokens e convertida para booleano
        mask_bool = self.mask.bool()[:num_tokens, :num_tokens]

        # Use a máscara para preencher os scores de atenção
        attn_scores.masked_fill_(mask_bool, -torch.inf)

        attn_weights = torch.softmax(attn_scores / keys.shape[-1]**0.5, dim=-1)
        attn_weights = self.dropout(attn_weights)

        # Shape: (b, num_tokens, num_heads, head_dim)
        context_vec = (attn_weights @ values).transpose(1, 2)

        # Combine as cabeças, onde self.d_out = self.num_heads * self.head_dim
        context_vec = context_vec.contiguous().view(b, num_tokens, self.d_out)
        context_vec = self.out_proj(context_vec) # optional projection

        return context_vec

torch.manual_seed(123)

batch_size, context_length, d_in = batch.shape
d_out = 2
mha = MultiHeadAttention(d_in, d_out, context_length, 0.0, num_heads=2)

context_vecs = mha(batch)

print(context_vecs)
print("context_vecs.shape:", context_vecs.shape)

tensor([[[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]],

        [[0.3190, 0.4858],
         [0.2943, 0.3897],
         [0.2856, 0.3593],
         [0.2693, 0.3873],
         [0.2639, 0.3928],
         [0.2575, 0.4028]]], grad_fn=<ViewBackward0>)
context_vecs.shape: torch.Size([2, 6, 2])


- Observe que o acima é essencialmente uma versão reescrita de `MultiHeadAttentionWrapper`, que é mais eficiente.  
- A saída resultante parece um pouco diferente, já que as inicializações de peso aleatórias diferem, mas ambas são implementações totalmente funcionais que podem ser usadas na classe GPT que implementaremos nos próximos capítulos.  
- Além disso, note que adicionamos uma camada de projeção linear (`self.out_proj`) à classe `MultiHeadAttention` acima. Isso é simplesmente uma transformação linear que não altera as dimensões. É uma convenção padrão usar tal camada de projeção na implementação de LLM, mas não é estritamente necessário (pesquisas recentes mostraram que ela pode ser removida sem afetar o desempenho do modelo; veja a seção de leituras adicionais no final deste capítulo).

<img src="https://sebastianraschka.com/images/LLMs-from-scratch-images/ch03_compressed/26.webp" width="400px">

- Note that if you are interested in a compact and efficient implementation of the above, you can also consider the [`torch.nn.MultiheadAttention`](https://pytorch.org/docs/stable/generated/torch.nn.MultiheadAttention.html) class in PyTorch

- Since the above implementation may look a bit complex at first glance, let's look at what happens when executing `attn_scores = queries @ keys.transpose(2, 3)`:

In [ ]:
# (b, num_heads, num_tokens, head_dim) = (1, 2, 3, 4)
a = torch.tensor([[[[0.2745, 0.6584, 0.2775, 0.8573],
                    [0.8993, 0.0390, 0.9268, 0.7388],
                    [0.7179, 0.7058, 0.9156, 0.4340]],

                   [[0.0772, 0.3565, 0.1479, 0.5331],
                    [0.4066, 0.2318, 0.4545, 0.9737],
                    [0.4606, 0.5159, 0.4220, 0.5786]]]])

print(a @ a.transpose(2, 3))

tensor([[[[1.3208, 1.1631, 1.2879],
          [1.1631, 2.2150, 1.8424],
          [1.2879, 1.8424, 2.0402]],

         [[0.4391, 0.7003, 0.5903],
          [0.7003, 1.3737, 1.0620],
          [0.5903, 1.0620, 0.9912]]]])


- In this case, the matrix multiplication implementation in PyTorch will handle the 4-dimensional input tensor so that the matrix multiplication is carried out between the 2 last dimensions (num_tokens, head_dim) and then repeated for the individual heads

- For instance, the following becomes a more compact way to compute the matrix multiplication for each head separately:

In [ ]:
first_head = a[0, 0, :, :]
first_res = first_head @ first_head.T
print("First head:\n", first_res)

second_head = a[0, 1, :, :]
second_res = second_head @ second_head.T
print("\nSecond head:\n", second_res)

First head:
 tensor([[1.3208, 1.1631, 1.2879],
        [1.1631, 2.2150, 1.8424],
        [1.2879, 1.8424, 2.0402]])

Second head:
 tensor([[0.4391, 0.7003, 0.5903],
        [0.7003, 1.3737, 1.0620],
        [0.5903, 1.0620, 0.9912]])


# Summary and takeaways

- See the [./multihead-attention.ipynb](./multihead-attention.ipynb) code notebook, which is a concise version of the data loader (chapter 2) plus the multi-head attention class that we implemented in this chapter and will need for training the GPT model in upcoming chapters
- You can find the exercise solutions in [./exercise-solutions.ipynb](./exercise-solutions.ipynb)